In [2]:
import os
from pyspark.sql import SparkSession

# Set environment variable so Java picks up your GCP credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "indigo-night-447804-g7-90004bf037f3.json"

# JARs required for BigQuery + GCS
jar_bigquery = "spark-bigquery-latest_2.12.jar"
jar_gcs = "gcs-connector-hadoop3-latest.jar"

# Initialize SparkSession with jars
spark = SparkSession.builder \
    .appName("UploadParquetToBigQuery") \
    .config("spark.jars", f"{jar_bigquery},{jar_gcs}") \
    .getOrCreate()

spark._jsc.hadoopConfiguration().set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
spark._jsc.hadoopConfiguration().set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
spark._jsc.hadoopConfiguration().set("fs.gs.auth.service.account.json.keyfile", "indigo-night-447804-g7-90004bf037f3.json")
spark._jsc.hadoopConfiguration().set("fs.gs.auth.service.account.enable", "true")


gcs_parquet_path = "gs://project_data_millionmusic/million_song_lake/part-00000-009fb375-9724-4b99-8069-9a3e450dd29a-c000.snappy.parquet"

# Read the Parquet file
df = spark.read.parquet(gcs_parquet_path)

# Show the first few rows (just to verify it loaded correctly)
df.show(5)

project_id = "indigo-night-447804-g7"
bq_dataset = "project_millionsongs"
bq_table = "millionsongsdata02"
temp_gcs_bucket = "project_data_millionmusic"

# Write DataFrame to BigQuery
df.write \
    .format("bigquery") \
    .option("table", f"{project_id}:{bq_dataset}.{bq_table}") \
    .option("temporaryGcsBucket", temp_gcs_bucket) \
    .mode("overwrite") \
    .save()

+--------------------+--------------------+--------------------+---------------+---------+----+--------+-------------------+-------------------+----------------------+
|            track_id|               title|         artist_name|artist_location| duration|year|loudness|  artist_hotttnesss|    song_hotttnesss|artist_hotttnesss_band|
+--------------------+--------------------+--------------------+---------------+---------+----+--------+-------------------+-------------------+----------------------+
|b'TRARRZU128F4253...|Je Sais Que La Te...|             Raphaël|         France|148.74077|2008|  -9.636| 0.3861516314132549| 0.5479529419800353|                Medium|
|b'TRARRJL128F92DE...|           On Efface|       Julie Zenatti|           null|252.99546|2004| -11.061|0.43485958934341257|0.47563846801023907|                Medium|
|b'TRARRUZ128F9307...|     Howells Delight|The Baltimore Con...|           null|  78.0273|   0|  -24.14|                0.0|               null|                